## Riot Games API

In [49]:
import requests
import json
import time
from tqdm import tqdm

In [36]:
challenger_url = 'https://euw1.api.riotgames.com/tft/league/v1/challenger?api_key=RGAPI-3baad8e8-f01e-432a-98bb-81d4639afd0a'
challenger_response = requests.get(url = challenger_url).json()

In [37]:
grandmaster_url = 'https://euw1.api.riotgames.com/tft/league/v1/grandmaster?api_key=RGAPI-3baad8e8-f01e-432a-98bb-81d4639afd0a'
grandmaster_response = requests.get(url = grandmaster_url).json()

In [40]:
with open('features/data/challenger.json', 'w') as json_file:
  json.dump(challenger_response, json_file)
with open('features/data/grandmaster.json', 'w') as json_file:
  json.dump(grandmaster_response, json_file)

In [51]:
challenger_df = pd.DataFrame(challenger_response['entries'])

puuidList = []
for x in tqdm(challenger_df['summonerId']):
    puuidUrl = f'https://euw1.api.riotgames.com/tft/summoner/v1/summoners/{x}?api_key=RGAPI-3baad8e8-f01e-432a-98bb-81d4639afd0a'
    puuidList.append(requests.get(url = puuidUrl).json())
    time.sleep(1.2)

100%|██████████| 144/144 [03:23<00:00,  1.41s/it]


In [52]:
with open('features/data/puuid.json', 'w') as json_file:
  json.dump(puuidList, json_file)

In [54]:
matchid_url = 'https://europe.api.riotgames.com/tft/match/v1/matches/by-puuid/HpQFceD22QJ4T0lfy-LyjRgZUBUMSlwZ9d0S0eUq9BhLPzUnhxrffZyOoyCrRjlyNlXxSjyucqSKsg/ids?count=20&api_key=RGAPI-3baad8e8-f01e-432a-98bb-81d4639afd0a'
matchid = requests.get(matchid_url).json()

In [56]:
with open('features/data/matchid.json', 'w') as json_file:
  json.dump(matchid, json_file)

In [57]:
match_info_list = []
for x in tqdm(matchid):
    match_info_url = f'https://europe.api.riotgames.com/tft/match/v1/matches/{x}?api_key=RGAPI-3baad8e8-f01e-432a-98bb-81d4639afd0a'
    match_info = requests.get(match_info_url).json()
    match_info_list.append(match_info)

100%|██████████| 20/20 [00:15<00:00,  1.29it/s]


In [58]:
with open('features/data/matchinfo.json', 'w') as json_file:
  json.dump(match_info_list, json_file)

## Stored JSON files

In [1]:
import pandas as pd
import json

/Users/yeungadrian/Documents/repo/ML-Ops/venv/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


### User info

In [4]:
with open('features/data/challenger.json', 'r') as myfile:
    challenger_league = json.loads(myfile.read())

challenger = pd.DataFrame(challenger_league['entries'])
challenger['tier'] = challenger_league['tier']
challenger['queue'] = challenger_league['queue']
challenger['name'] = challenger_league['name']
challenger['leagueId'] = challenger_league['leagueId']

In [51]:
with open('features/data/puuid.json', 'r') as myfile:
    userinfo = json.loads(myfile.read())

userinfo = pd.DataFrame(userinfo)
userinfo.rename(columns={'id':'summonerId'}, inplace=True)


In [57]:
final = userinfo.merge(challenger, on='summonerId', how='outer', suffixes=('', '_y'))
final.drop(final.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)

In [60]:
final.head()

,summonerId,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak,tier,queue,leagueId
0,LscjZAcPTKkFs7Dc7dfw8UDEwEgYyGqfl_xqOeFw4wNJUjQM,NxGeXxzfporSC_EMbaPhm2SM8vfU_k3Vlnl12-DmEjHFIb...,HpQFceD22QJ4T0lfy-LyjRgZUBUMSlwZ9d0S0eUq9BhLPz...,Rubic TFT,3372,1628365040000,1,Rubic TFT,625,I,18,68,False,False,False,False,CHALLENGER,RANKED_TFT,d4c40d11-deaa-39aa-89e1-8b6ab6c0e54b
1,MVXf_l367lUhxEcxIfp22NDUS4Ik9ySn8osNecnPB6zLXlY,5vH6zGfGVuc5c1IPb-hLA17D92BrwOOR7g2SppwY1PIvCyM,yWoHQzvaGUwz3C2X1RDIkJxR46uR2Na0myHOAQWLvUNv4N...,SwellerTiger,682,1628448759000,29,SwellerTiger,1478,I,26,170,True,False,False,False,CHALLENGER,RANKED_TFT,d4c40d11-deaa-39aa-89e1-8b6ab6c0e54b
2,iMgDLIY6msbypDOyvOJ-zsUTDQoivFw4_NHcL4PA_uWAQqE,nBaYo05HyPmI0g69Zyrsic4Z8MXqs6PGLFvwsLwftwfCZUU,pRafrNcjVt3t20jQwkxf47uPGpOdXkIOSxDUcaDU2S-Pje...,KRC Lilith,5021,1628460985000,308,KRC Lilith,829,I,16,100,False,False,False,False,CHALLENGER,RANKED_TFT,d4c40d11-deaa-39aa-89e1-8b6ab6c0e54b
3,eBt7hXz1mbp85ZppRCZktthn9Slvi22b0dL0HOXTbDpIG-M,YnOoVlcevhh_QtZZKNaGNeD1FftbMdt56l2sHXSSig9zTA,hw-KA4_LwT2DmK45uzZeFOAsMr8JMNe6jfz4pNzLXlXeTV...,Maxwellβoltzman,4448,1628498755000,159,Maxwellβoltzman,907,I,23,62,False,False,False,False,CHALLENGER,RANKED_TFT,d4c40d11-deaa-39aa-89e1-8b6ab6c0e54b
4,1KUoyc3kTwN0D_EXUGgjEHjBmVOpwhmZnnlwADUNeyhfu4nl,iij91U4ULDphRX2_DKsWp9poM3Sqzz_Vj_iyTFuuJWLw4k...,w9ZzinwNyH8Tls9Qz7Hd1oyqm3ct1NIuy6Pkvgycy_snx2...,AL Specter,4782,1628275511000,46,AL Specter,547,I,11,109,False,False,True,False,CHALLENGER,RANKED_TFT,d4c40d11-deaa-39aa-89e1-8b6ab6c0e54b


### Match Info

In [61]:
with open('features/data/matchinfo.json', 'r') as myfile:
    matchinfo = json.loads(myfile.read())

In [64]:
pd.DataFrame(matchinfo[0]['metadata'])

,data_version,match_id,participants
0,5,EUW1_5403919165,wicN6Iue2SYJQodBh8PI9RpEOzAhSxNDCg9_fSrvTVViSs...
1,5,EUW1_5403919165,Z5NUEosvgcvnaFr4r2ia67AkTzw5xxmOwwFBtce5L7obmF...
2,5,EUW1_5403919165,zh2iXAtCkd2dwcGskgmby0Ka7Vb0hK058vFv1mL_HTYDa1...
3,5,EUW1_5403919165,Z2dlM1CgIEQbrFjswkXhh3vRf-0u7oMIbY637zwdFP265O...
4,5,EUW1_5403919165,ePWIL0nsi0S8VrZR6bbUdV-IM6HWpRczZ7DoEui0SY_JY1...
5,5,EUW1_5403919165,2CfFSIJJMqD8jXSaI4mkxnaIbKnHzgqdZ7SeQdcBpusDO9...
6,5,EUW1_5403919165,HpQFceD22QJ4T0lfy-LyjRgZUBUMSlwZ9d0S0eUq9BhLPz...
7,5,EUW1_5403919165,FjSMpwGp7vGk3fLAmz8GJ75rbSR7EhzILoOZD3GJAAOL9X...


In [65]:
pd.DataFrame(matchinfo[0]['info'])

,game_datetime,game_length,game_version,participants,queue_id,tft_game_type,tft_set_number
0,1628363151284,2025.280029,Version 11.15.389.2308 (Jul 26 2021/13:45:59) ...,{'companion': {'content_ID': '83ca07f4-8572-45...,1100,standard,5
1,1628363151284,2025.280029,Version 11.15.389.2308 (Jul 26 2021/13:45:59) ...,{'companion': {'content_ID': 'e11fc781-0df7-40...,1100,standard,5
2,1628363151284,2025.280029,Version 11.15.389.2308 (Jul 26 2021/13:45:59) ...,{'companion': {'content_ID': '809198fc-3115-43...,1100,standard,5
3,1628363151284,2025.280029,Version 11.15.389.2308 (Jul 26 2021/13:45:59) ...,{'companion': {'content_ID': '5f3afd9b-a2a7-4c...,1100,standard,5
4,1628363151284,2025.280029,Version 11.15.389.2308 (Jul 26 2021/13:45:59) ...,{'companion': {'content_ID': '8c304205-ce41-4f...,1100,standard,5
5,1628363151284,2025.280029,Version 11.15.389.2308 (Jul 26 2021/13:45:59) ...,{'companion': {'content_ID': 'bdb8527c-baa5-43...,1100,standard,5
6,1628363151284,2025.280029,Version 11.15.389.2308 (Jul 26 2021/13:45:59) ...,{'companion': {'content_ID': '06b621fc-73ec-49...,1100,standard,5
7,1628363151284,2025.280029,Version 11.15.389.2308 (Jul 26 2021/13:45:59) ...,{'companion': {'content_ID': '8c27f67c-90b3-43...,1100,standard,5


In [66]:
pd.DataFrame(matchinfo[0]['info']['participants'])

,companion,gold_left,last_round,level,placement,players_eliminated,puuid,time_eliminated,total_damage_to_players,traits,units
0,{'content_ID': '83ca07f4-8572-45ae-97df-55492b...,1,31,7,5,0,wicN6Iue2SYJQodBh8PI9RpEOzAhSxNDCg9_fSrvTVViSs...,1714.410034,74,"[{'name': 'Set5_Abomination', 'num_units': 1, ...","[{'character_id': 'TFT5_Pyke', 'items': [], 'n..."
1,{'content_ID': 'e11fc781-0df7-4087-acd1-57264a...,0,27,6,8,0,Z5NUEosvgcvnaFr4r2ia67AkTzw5xxmOwwFBtce5L7obmF...,1498.000610,41,"[{'name': 'Set5_Cannoneer', 'num_units': 2, 's...","[{'character_id': 'TFT5_Kled', 'items': [], 'n..."
2,{'content_ID': '809198fc-3115-4375-89f4-1c1a1d...,0,28,8,7,0,zh2iXAtCkd2dwcGskgmby0Ka7Vb0hK058vFv1mL_HTYDa1...,1564.103882,93,"[{'name': 'Set5_Abomination', 'num_units': 3, ...","[{'character_id': 'TFT5_Brand', 'items': [47],..."
3,{'content_ID': '5f3afd9b-a2a7-4c5c-858a-68c1d3...,2,37,8,1,3,Z2dlM1CgIEQbrFjswkXhh3vRf-0u7oMIbY637zwdFP265O...,2017.081909,161,"[{'name': 'Set5_Abomination', 'num_units': 1, ...","[{'character_id': 'TFT5_Lulu', 'items': [], 'n..."
4,{'content_ID': '8c304205-ce41-4fd6-ae94-8623e8...,0,34,8,4,0,ePWIL0nsi0S8VrZR6bbUdV-IM6HWpRczZ7DoEui0SY_JY1...,1868.147705,120,"[{'name': 'Set5_Abomination', 'num_units': 3, ...","[{'character_id': 'TFT5_Syndra', 'items': [46]..."
5,{'content_ID': 'bdb8527c-baa5-4378-90a6-9a54f2...,0,37,8,2,2,2CfFSIJJMqD8jXSaI4mkxnaIbKnHzgqdZ7SeQdcBpusDO9...,2011.708862,123,"[{'name': 'Set5_Abomination', 'num_units': 1, ...","[{'character_id': 'TFT5_Khazix', 'items': [207..."
6,{'content_ID': '06b621fc-73ec-4932-b091-93a7f5...,1,28,8,6,0,HpQFceD22QJ4T0lfy-LyjRgZUBUMSlwZ9d0S0eUq9BhLPz...,1565.084595,71,"[{'name': 'Set5_Brawler', 'num_units': 1, 'sty...","[{'character_id': 'TFT5_Vladimir', 'items': []..."
7,{'content_ID': '8c27f67c-90b3-4355-9e90-6e9232...,8,37,8,3,1,FjSMpwGp7vGk3fLAmz8GJ75rbSR7EhzILoOZD3GJAAOL9X...,2012.690552,125,"[{'name': 'Set5_Abomination', 'num_units': 3, ...","[{'character_id': 'TFT5_Brand', 'items': [9, 4..."


In [70]:
pd.DataFrame(matchinfo[0]['info']['participants'][0]['traits'])

,name,num_units,style,tier_current,tier_total
0,Set5_Abomination,1,0,0,3
1,Set5_Assassin,4,3,2,3
2,Set5_Invoker,1,0,0,2
3,Set5_Mystic,1,0,0,4
4,Set5_Nightbringer,2,1,1,4
5,Set5_Renewer,1,0,0,3
6,Set5_Revenant,3,2,2,4
7,Set5_Sentinel,1,0,0,3


In [71]:
pd.DataFrame(matchinfo[0]['info']['participants'][0]['units'])

,character_id,items,name,rarity,tier
0,TFT5_Pyke,[],,1,2
1,TFT5_Nocturne,"[49, 1178, 26]",Nocturne,2,2
2,TFT5_Nocturne,"[29, 11, 2022]",Nocturne,2,2
3,TFT5_Diana,[],,3,1
4,TFT5_Ivern,[],Ivern,3,1
5,TFT5_Fiddlesticks,"[89, 33, 12]",,3,1
6,TFT5_Fiddlesticks,[],,3,1


In [6]:
import psycopg2
from config import config
import pandas as pd

In [10]:
params = config()
# Connect to the PostgreSQL database
conn = psycopg2.connect(**params)
# Create a new cursor
cur = conn.cursor()

# A function that takes in a PostgreSQL query and outputs a pandas database 
def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

In [11]:
vendor_info = create_pandas_table("SELECT vendor_id, vendor_name FROM vendors ORDER BY vendor_name")
vendor_info

NameError: name 'pd' is not defined